In [1]:
import os
os.chdir('../')

In [2]:
import snappy

from snappy import ProductIO
from snappy import HashMap

import os, gc   
from snappy import GPF

GPF.getDefaultInstance().getOperatorSpiRegistry().loadOperatorSpis()
HashMap = snappy.jpy.get_type('java.util.HashMap')


In [3]:
"""
Input:  Amplitude_VH, Intesity_VH, Amplitude_VV, Intensity_VV
output: Amplitude_VH, Intesity_VH, Amplitude_VV, Intensity_VV
"""
def applyOrbit(timestamp, sentinel_image, HashMap):
    print("applying orbit")
    file_format = '.dim'
    path_output = os.path.join(input_dir, "orb_" + timestamp + file_format)
    
    if os.path.isfile(path_output):
        print "file already exists"
    else:
        
        parameters = HashMap()
        orbit_param = snappy.GPF.createProduct("Apply-Orbit-File", parameters, sentinel_image)
        ProductIO.writeProduct(orbit_param, path_output[:-4], 'BEAM-DIMAP')  
        
    return 0


def applyThermalNoiseRemoval(timestamp, HashMap):

    print('applying Thermal noise removal')
    file_format = '.dim'
    path_output = os.path.join(input_dir, "therm_" + timestamp + file_format)
    path_input = os.path.join(input_dir, "orb_" +  timestamp + file_format)

    if os.path.isfile(path_output):
        print "file already exists"   
    else:       
        parameters = HashMap()
        parameters.put('removeThermalNoise', True)
        thermal_param = snappy.GPF.createProduct('ThermalNoiseRemoval', parameters, ProductIO.readProduct(path_input))

        ProductIO.writeProduct(thermal_param, path_output[:-4], 'BEAM-DIMAP')
    
    return 0

"""
Input:  Amplitude_VH, Intesity_VH, Amplitude_VV, Intensity_VV
output: beta0_VH, beta0_VV
"""
def applyCalibration(timestamp, HashMap):
    print("applying calibration")
    file_format = '.dim' 
    path_output = os.path.join(input_dir, "calib_" + timestamp + file_format)
    path_input = os.path.join(input_dir, "therm_" + timestamp + file_format)

    if os.path.isfile(path_output):
        print "file already exists"
    else:
        parameters = HashMap()
        parameters.put('outputSigmaBand', False)
        parameters.put('outputBetaBand', True)

        calib_param = snappy.GPF.createProduct("Calibration", parameters, ProductIO.readProduct(path_input))
        ProductIO.writeProduct(calib_param, path_output[:-4], 'BEAM-DIMAP')
    
    return 0
 
"""
Input:  beta0_VH, beta0_VV
output: beta0_VH, beta0_VV
"""
def applyMultilook(timestamp, HashMap):
 
    print("applying mulilook")
    file_format = '.dim'
    path_output = os.path.join(input_dir, "multi_" + timestamp + file_format) 
    path_input = os.path.join(input_dir, "calib_" + timestamp + file_format)

    if os.path.isfile(path_output):
        print "file already exisist"       

    else:
        azLooks = 3
        rgLooks = 3
        parameters = HashMap()

        parameters.put('grSquarePixel', True)
        parameters.put('nRgLooks', rgLooks)
        parameters.put('nAzLooks', azLooks)
        parameters.put('outputIntensity', False)
        multi_param = snappy.GPF.createProduct("Multilook", parameters, ProductIO.readProduct(path_input))
        ProductIO.writeProduct(multi_param, path_output[:-4], 'BEAM-DIMAP')
        
    return 0
 
"""
Input:  beta0_VH, beta0_VV
output: gamma0_VH, gamma0_VV
"""
def applyTerrainCorrection(timestamp, HashMap):

    print("applying terrain correction")
    file_format = '.dim'

    path_output = os.path.join(input_dir, "terr_" + timestamp + file_format)
    path_input = os.path.join(input_dir, "multi_" +  timestamp + file_format)

    if os.path.isfile(path_output):
        print "file already exists"        

    else:
        parameters = HashMap()

        parameters.put('demResamplingMethod', 'BICUBIC_INTERPOLATION')
        parameters.put('demName', 'SRTM 1Sec HGT')

        terr_param = snappy.GPF.createProduct("Terrain-Flattening", parameters,ProductIO.readProduct(path_input))
        ProductIO.writeProduct(terr_param, path_output[:-4], 'BEAM-DIMAP')
    
    return 0

"""
Input:  gamma0_VH, gamma0_VV
output: gamma0_VH, gamma0_VV
"""
def applyOrthorectification(timestamp, HashMap):
    
    print("applying Orthorectification")
    file_format = '.dim'
    
    path_output = os.path.join(input_dir, "orth_" + timestamp + file_format)
    path_input = os.path.join(input_dir, "terr_" +  timestamp + file_format)

    if os.path.isfile(path_output):
        print "file already exists"        

    else:
        parameters = HashMap()

        orth_param = snappy.GPF.createProduct("Ellipsoid-Correction-GG", parameters,ProductIO.readProduct(path_input))
        ProductIO.writeProduct(orth_param, path_output[:-4], 'BEAM-DIMAP')
    
    return 0

"""
Input:  gamma0_VH, gamma0_VV
output: gamma0_VH, gamma0_VV
"""
def applySpeckleFilter(timestamp, HashMap):
    
    filter = 7
    print("applying Speckle filter")
    file_format = '.dim'
    path_output = os.path.join(input_dir, "speck_" + timestamp + file_format)
    path_input = os.path.join(input_dir, "orth_" +  timestamp + file_format)

    if os.path.isfile(path_output):
        print "file already exists"        

    else:
        parameters = HashMap()

        parameters.put('filter', 'Lee Sigma')
        parameters.put('numberofLooks', 4)
        parameters.put('windowSize', "{}x{}".format(filter, filter))
        parameters.put('sigma', 0.9)
        parameters.put('targetWindowSize', "{}x{}".format(filter, filter))

        speck_param = snappy.GPF.createProduct("Speckle-Filter", parameters,ProductIO.readProduct(path_input))
        ProductIO.writeProduct(speck_param, path_output[:-4], 'GeoTIFF')

    return 0

In [4]:

folder = 'data/processed/S1A_IW_GRDH_1SDV_20201222T174529_20201222T174554_035802_0430E6_8C1D.SAFE'
input_dir = 'data/processed/'

timestamp = folder.split("_")[5]
sentinel_image = ProductIO.readProduct(folder)

In [5]:
%%time
# add the orbit file
applyOrbit(timestamp, sentinel_image, HashMap)

applying orbit
CPU times: user 28.4 s, sys: 4.14 s, total: 32.6 s
Wall time: 1min 4s


0

In [6]:
%%time
# Thermal Noise Removal
applyThermalNoiseRemoval(timestamp, HashMap)

applying Thermal noise removal
CPU times: user 1min 8s, sys: 7.29 s, total: 1min 15s
Wall time: 3min 18s


0

In [7]:
%%time
# Calibration
applyCalibration(timestamp, HashMap)

applying calibration
CPU times: user 46.3 s, sys: 8.99 s, total: 55.3 s
Wall time: 4min 14s


0

In [8]:
%%time
# Multi look
applyMultilook(timestamp, HashMap)

applying mulilook
CPU times: user 16.9 s, sys: 7.11 s, total: 24.1 s
Wall time: 3min 10s


0

In [9]:
%%time
# Terrain Correction
applyTerrainCorrection(timestamp, HashMap)

applying terrain correction
CPU times: user 6min 44s, sys: 1.41 s, total: 6min 46s
Wall time: 10min 6s


0

In [10]:
%%time
# Ortho rectification
applyOrthorectification(timestamp, HashMap)

applying Orthorectification
CPU times: user 3min 36s, sys: 1.41 s, total: 3min 37s
Wall time: 2min 51s


0

In [11]:
%%time
# Speckle Filter
applySpeckleFilter(timestamp, HashMap)

applying Speckle filter
CPU times: user 4min 22s, sys: 3.43 s, total: 4min 26s
Wall time: 3min 24s


0

In [ ]:
"""
Orbit files may be downloaded from https://qc.sentinel1.eo.esa.int/
and placed in /home/ankush/.snap/auxdata/Orbits/Sentinel-1/POEORB/S1A/2020/12

"""